# Querying the Graph to study the universe. 


Eventually this will be an admin page. For now I'm just researching it. 


In [2]:
import sys, os
import pandas as pd
import altair as alt
sys.path.append('..')

import  ssl, asyncio

ssl._create_default_https_context = ssl._create_unverified_context
asyncio.set_event_loop_policy(asyncio.WindowsSelectorEventLoopPolicy())
import nest_asyncio
# this is required for running in a Jupyter Notebook. 

nest_asyncio.apply()

PII_cols = ['username','userguid','owner']

Most of the `asyncio` and `ssl` stuff is only used for notebooks, it's just here to demo the module. 

In [3]:
from app.connectors.cmdb_graph import CosmosdbClient

executing local windows deployment


connecting using the same module in the app. 

In [4]:
c = CosmosdbClient()


Testing the co

In [5]:
c.run_query()
c.res

[8120]

## Looking at a particular planet

In [18]:
location_id = "9546786684762"
planet_query = f"g.V().has('objid', '{location_id}').valuemap()"
print(planet_query)
c.run_query(planet_query)
c.clean_nodes(c.res)

g.V().has('objid', '9546786684762').valuemap()


[{'name': 'Heimsan',
  'class': 'terrestrial',
  'objid': '9546786684762',
  'pop_cap': 100,
  'atmosphere': '[{Argon: 0.024}, {Carbon Dioxide: 0.019}, {Helium: 0.022}, {Hydrogen: 0.36}, {Methane: 0.005}, {Nitrogen: 0.28}, {Oxygen: 0.183}, {Sodium: 0.106}]',
  'radius': 0.528,
  'mass': 0.316,
  'orbitsDistance': 0.426,
  'orbitsId': '6261400091664',
  'orbitsName': 'Nal',
  'isSupportsLife': 'true',
  'isPopulated': 'true',
  'isHomeworld': 'true',
  'userguid': '8d5b667f-b225-4641-b499-73b77558ff86',
  'objtype': 'planet',
  'id': '9546786684762'}]

In [19]:
meta_count = f"g.V().has('objid', '{location_id}').out().groupcount().by('label')"


c.run_query(meta_count)
c.res

[{'star': 1, 'system': 1, 'resource': 4}]

In [20]:
meta_count = f"g.V().has('objid', '{location_id}').in().groupcount().by('label')"


c.run_query(meta_count)
c.res

[{'moon': 2, 'pop': 101, 'biome': 1}]

In [23]:
meta_count = f"g.V().has('objid', '{location_id}').out('has').valuemap()"
c.run_query(meta_count)
c.res

[{'volume': [0],
  'name': ['organics'],
  'objid': ['2111521261896'],
  'max_volume': [1003],
  'description': ['bilogical material that can be consumed by pops'],
  'replenish_rate': [10],
  'userguid': ['8d5b667f-b225-4641-b499-73b77558ff86'],
  'objtype': ['resource']},
 {'name': ['common minerals'],
  'objid': ['2953562742312'],
  'volume': [88],
  'max_volume': [88],
  'description': ['Iron and other common material used in constructuion of infrastructure'],
  'userguid': ['8d5b667f-b225-4641-b499-73b77558ff86'],
  'objtype': ['resource']},
 {'name': ['rare minerals'],
  'objid': ['3990725193104'],
  'volume': [52],
  'max_volume': [52],
  'description': ['lithium, silver and other rare minerals used in infrastructure and technology'],
  'userguid': ['8d5b667f-b225-4641-b499-73b77558ff86'],
  'objtype': ['resource']},
 {'name': ['water'],
  'objid': ['6104225671322'],
  'volume': [9735],
  'max_volume': [9735],
  'description': ['H2O ready to be consumed, either frozen or in ice 

Ok. So why aren't they starving. 

In [24]:
meta_count = f"g.V().has('label', 'action').valuemap()"
c.run_query(meta_count)
c.res

[{'type': ['individual_education'],
  'requires_attr': ['{wealth: 0.1}'],
  'applies_to': ['pop'],
  'effort': [2],
  'augments_self_properties': ['{wealth: -.1, literacy: 0.1, factionLoyalty: -.05}'],
  'comment': ['expand privatized education, increase literacy and wealth at the cost of faction loyalty'],
  'name': ['individual education'],
  'created_at': [1014],
  'objid': ['7161877396226'],
  'username': ['BillmanLocal2'],
  'objtype': ['action']},
 {'type': ['construction'],
  'comment': ['constructing a farmland'],
  'effort': [1],
  'applies_to': ['pop'],
  'owned_by': ['pop'],
  'building': ['farmland'],
  'created_at': [1014],
  'objid': ['1021261158046'],
  'username': ['BillmanLocal2'],
  'objtype': ['action']},
 {'type': ['construction'],
  'comment': ['constructing a solar panel'],
  'effort': [3],
  'applies_to': ['pop'],
  'owned_by': ['pop'],
  'building': ['solar_panel'],
  'created_at': [1014],
  'objid': ['0302107294481'],
  'username': ['BillmanLocal2'],
  'objtype